A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [11]:
using Globtim
# include("../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 8 # Initial Degree 
const n, a, b = 2, 1, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-3             # Define the tolerance for the L2-norm

N = 3
params = init_gaussian_params(n, N, .2)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized


In [ ]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end;

In [ ]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d)) # Convert coefficients to Float64 for homotopy continuation
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)
println("The system is of degree:", d-1)

In [ ]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

In [15]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [ ]:
using PlotlyJS, Colors

coords = poly_approx.scale_factor * poly_approx.grid # Extract coordinates and function values
z_coords = poly_approx.z

if size(coords)[2] == 2 # Plot the 3D scatter plot if the dimensions are 2
    scatter_trace = scatter3d(
        x=coords[:, 1],
        y=coords[:, 2],
        z=z_coords,
        mode="markers",
        marker=attr(
            size=1,
            color=z_coords,
            colorscale="Viridis"
        ),
        name="Sampled Data"
    )

    # Create the scatter3d trace
    crit_pts = scatter3d(
        x=df.x,
        y=df.y,
        z=df.z,
        mode="markers",
        marker=attr(
            size=10,
            color="red"
        ),
        name="Exact approximant critical points"
    )

    layout = Layout(
        title="Mixture of Gaussians Sample Points",
        scene=attr(
            xaxis=attr(title="X-axis"),
            yaxis=attr(title="Y-axis"),
            zaxis=attr(title="Z-axis")),
        height=1200
    )
    plt1 = Plot([scatter_trace, crit_pts], layout)
    # display(plt1)
end


In [ ]:
# Optimize the collected entries 
using Optim
for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    println(summary(res))
end


In [ ]:
sf = surface(x=x, y=y, z=z)
# Had to switch the coordinates of the critical points to match the surface plot for some reason. 
crit_pts = scatter3d(x=df.y, y=df.x, z=df.z, mode="markers", marker_size=5, name="Critical Points")

# Layout for the plot
layout = Layout(title="3D Plot of Shubert function with the critical points of the approximant of degree 22",
    scene=attr(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x, y)"),
    height=800
)
# Display the plot layout,
plt = plot([sf, crit_pts], layout)

In [19]:
# savefig(plt, "../data/figures/random_gaussian_function_plot.html")

Should plot the polynomial approximant too.